## Required Libraries 

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

from typing import List

from matplotlib import cm
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn
import time
from tqdm import tqdm
from scipy.optimize import curve_fit

from IPython import display
from scipy.stats import pearsonr

In [ ]:

    
    
    
    



    # if not ep % 500:      
    #     D, T2, v = model.encode(test_tensor)

    #     #fig, ax = plt.subplots(3,3, figsize=(15,15))
    #     for r in range(3):
    #         if r==0:
    #             x_image, y_image = v_test, v
    #             title = ['V_ep', 'V st', 'V lu']
    #         elif r==1:
    #             x_image, y_image = D_test2, D
    #             title = ['D ep', 'D st', 'D lu']
    #         elif r==2:
    #             x_image, y_image = T2_test2, T2
    #             title = ['T2 ep', 'T2 st', 'T2 lu']
    #         for c in range(3):

    #             x = x_image.detach().cpu().numpy()[:,c] 
    #             y = y_image[:, c].detach().cpu().numpy()

    #             corr = round(pearsonr(y, x)[0], 2)
    #             mae = np.mean(np.abs(y - x))
    #             bias = np.mean((y - x))
    #             std = np.std((y - x))
    #             print(f'{title[c]} {corr:.2f} {mae:.2f} {bias:.2f} {std:.2f}')

                #nbins=300
                #k = gaussian_kde([x,y])
                #xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
                #zi = k(np.vstack([xi.flatten(), yi.flatten()]))
                #ax[r,c].pcolormesh(xi, yi, zi.reshape(xi.shape), cmap="hot", shading='auto')
                #ax[r,c].scatter(x,y, color='white', s=8, alpha=0.5)

                #err = np.mean(np.abs(x-y))
                #corr = np.corrcoef(x,y)[0,1]
                #print(fr'{title[c]}, MAE = {err:.3f}, $\rho$ = {corr:.3f}')
                #ax[r,c].set_title(fr'{title[c]}, MAE = {err:.3f}, $\rho$ = {corr:.3f}')
                #ax[r,c].set_xlabel('true', fontsize=20)
                #ax[r,c].set_ylabel('predicted', fontsize=20)
        #display.display(plt.gcf())


#     models.append(model)
#     print(f'Model {trial + 1} trained')
#     D, T2, v = 0, 0, 0
#     for model in models:
#         model.eval()
#         D_, T2_, v_ = model.encode(test_tensor)
#         D += D_
#         T2 += T2_
#         v += v_
#     D /= len(models)
#     T2 /= len(models)
#     v /= len(models)
#     for r in range(3):
#         if r==0:
#             x_image, y_image = v_test, v
#             title = ['V_ep', 'V st', 'V lu']
#         elif r==1:
#             x_image, y_image = D_test2, D
#             title = ['D ep', 'D st', 'D lu']
#         elif r==2:
#             x_image, y_image = T2_test2, T2
#             title = ['T2 ep', 'T2 st', 'T2 lu']
#         for c in range(3):

#             x = x_image.detach().cpu().numpy()[:,c] 
#             y = y_image[:, c].detach().cpu().numpy()

#             corr = round(pearsonr(y, x)[0], 2)
#             mae = np.mean(np.abs(y - x))
#             bias = np.mean((y - x))
#             std = np.std((y - x))
#             print(f'{title[c]} {corr:.2f} {mae:.2f} {bias:.2f} {std:.2f}')



In [ ]:
D, T2, v = model.encode(test_tensor)

#fig, ax = plt.subplots(3,3, figsize=(15,15))
for r in range(3):
    if r==0:
        x_image, y_image = v_test, v
        title = ['V_ep', 'V st', 'V lu']
    elif r==1:
        x_image, y_image = D_test2, D
        title = ['D ep', 'D st', 'D lu']
    elif r==2:
        x_image, y_image = T2_test2, T2
        title = ['T2 ep', 'T2 st', 'T2 lu']
    for c in range(3):

        x = x_image.detach().cpu().numpy()[:,c] 
        y = y_image[:, c].detach().cpu().numpy()

        corr = round(pearsonr(y, x)[0], 2)
        mae = np.mean(np.abs(y - x))
        bias = np.mean((y - x))
        std = np.std((y - x))
        print(f'{title[c]} {corr:.2f} {mae:.4f} {bias:.4f} {std:.4f}')


In [ ]:
def get_scores(test, pia, hybrid, inx):    
    x1 = test.detach().cpu().numpy()[:,inx] 
    y1 = pia[:, inx].detach().cpu().numpy()
    corr1 = round(pearsonr(y1, x1)[0], 2)
    mae1 = np.mean(np.abs(y1 - x1))
    bias1 = np.mean((y1 - x1))
    std1 = np.std((y1 - x1))
    x2 = test.detach().cpu().numpy()[:, inx] 
    y2 = hybrid[:, inx]
    corr2 = round(pearsonr(y2, x2)[0], 2)
    mae2 = np.mean(np.abs(y2 - x2))
    bias2 = np.mean((y2 - x2))
    std2 = np.std((y2 - x2))
    return  (corr1, corr2), (mae1,mae2), (bias1,bias2), (std1,std2)
    
PATH = '/home/gundogdu/Desktop/PIA_Histology/pia_model.pt'
model = torch.load(PATH)
noise_levels = [2*0.0001, 5*0.0001, 7*0.0001, 0.001, 2*0.001, 5*0.001, 7*0.001, 0.01, 0.02, 0.05, 0.07, 0.1]
v_ep = []
v_st = []
v_lu = []

D_ep = []
D_st = []
D_lu = []
T2_ep = []
T2_st = []
T2_lu = []

for N in noise_levels:
    B = 1000
    _, D_test2, T2_test2, v_test, clean = get_batch(B, noise_sdt=0)
    clean = clean.detach().cpu().numpy()
    noise_im = np.random.normal(0, N, (B, 16))
    noise_re = np.random.normal(0, N, (B, 16))
    noisy = np.sqrt((clean + noise_im)**2 + (clean + noise_re)**2)/np.sqrt(2)

    test_tensor = 1000*torch.from_numpy(noisy).float()
    test_tensor = test_tensor.cuda()
    D_pia, T2_pia, v_pia = model.encode(test_tensor)
    test = test_tensor.detach().cpu().numpy()/1000
    D_hybrid, T2_hybrid, v_hybrid = hybrid_fit(test)
    corr, mae, bias, std = get_scores(v_test, v_pia, v_hybrid, 0)    
    v_ep.append({'corr' : corr, 'mae' : mae, 'bias' : bias, 'std' : std})
    corr, mae, bias, std = get_scores(v_test, v_pia, v_hybrid, 1)    
    v_st.append({'corr' : corr, 'mae' : mae, 'bias' : bias, 'std' : std})
    corr, mae, bias, std = get_scores(v_test, v_pia, v_hybrid, 2)    
    v_lu.append({'corr' : corr, 'mae' : mae, 'bias' : bias, 'std' : std})
    
    corr, mae, bias, std = get_scores(D_test2, D_pia, D_hybrid, 0)    
    D_ep.append({'corr' : corr, 'mae' : mae, 'bias' : bias, 'std' : std})
    corr, mae, bias, std = get_scores(D_test2, D_pia, D_hybrid, 1)    
    D_st.append({'corr' : corr, 'mae' : mae, 'bias' : bias, 'std' : std})
    corr, mae, bias, std = get_scores(D_test2, D_pia, D_hybrid, 2)    
    D_lu.append({'corr' : corr, 'mae' : mae, 'bias' : bias, 'std' : std})
    
    corr, mae, bias, std = get_scores(T2_test2, T2_pia, T2_hybrid, 0)    
    T2_ep.append({'corr' : corr, 'mae' : mae, 'bias' : bias, 'std' : std})
    corr, mae, bias, std = get_scores(T2_test2, T2_pia, T2_hybrid, 1)    
    T2_st.append({'corr' : corr, 'mae' : mae, 'bias' : bias, 'std' : std})
    corr, mae, bias, std = get_scores(T2_test2, T2_pia, T2_hybrid, 2)    
    T2_lu.append({'corr' : corr, 'mae' : mae, 'bias' : bias, 'std' : std})



In [ ]:
sequence_names = ['Ep. Vol.', 'St. Vol.', 'Lu. Vol' , 'Ep. D.', 'St. D.', 'Lu. D', 'Ep. T2.', 'St. T2.', 'Lu. T2']
measure_names = ['Pearson R', 'MAE', 'Bias', 'Std.Dev']
for name, sq in enumerate([v_ep, v_st, v_lu, D_ep, D_st, D_lu, T2_ep, T2_st, T2_lu]):
    for meas, kw in enumerate(['corr', 'mae', 'bias', 'std']):
        fig, ax = plt.subplots(1,1)
        P = [x[kw][0] for x in sq]
        H = [x[kw][1] for x in sq]
        line1 = ax.plot(noise_levels, P, color='skyblue', lw=2, marker='o', label='PIA')
        line2 = ax.plot(noise_levels, H, color='salmon', lw=2, marker='s', label ='NLLS')
        ax.set_xscale('log')
        ax.set_title(f'{sequence_names[name]} {measure_names[meas]}')
        ax.legend(['PIA', 'NLLS'])
        ax.grid(True)




In [ ]:
from scipy import stats
def steigers_z_test(r1, r2, n1, n2):
    """
    Performs Steiger's Z-test for two dependent correlation coefficients sharing one variable in common.

    Args:
    r1 (float): Pearson correlation coefficient for the first comparison.
    r2 (float): Pearson correlation coefficient for the second comparison.
    n1 (int): Sample size for the first comparison.
    n2 (int): Sample size for the second comparison.

    Returns:
    float: Z-score indicating the difference between the two correlation coefficients.
    float: p-value assessing the significance of the Z-score.
    """
    # Fisher Z transformation for each correlation
    z1 = np.arctanh(r1)
    z2 = np.arctanh(r2)

    # Standard error for each transformed correlation
    se1 = 1 / np.sqrt(n1 - 3)
    se2 = 1 / np.sqrt(n2 - 3)

    # Standard error of the difference
    sed = np.sqrt(se1**2 + se2**2)

    # Z-score
    z = (z1 - z2) / sed

    # Two-tailed p-value
    p = 2 * (1 - stats.norm.cdf(np.abs(z)))

    return z, p

def calculate_mae_bias_variance(y_true, y_pred):
    """
    Calculate the Mean Absolute Error (MSE) and Bias.

    Args:
    y_true (array-like): Actual values.
    y_pred (array-like): Predicted or estimated values.

    """
    mae = np.mean(np.abs(y_true - y_pred))
    bias = np.mean((y_pred - y_true))
    variance = np.std((y_pred - y_true))**2

    return mae, bias, variance

def compare_mae(y_true, y_pred1, y_pred2):

    mae1, bias1, variance1 = calculate_mae_bias_variance(y_true, y_pred1)
    mae2, bias2, variance2 = calculate_mae_bias_variance(y_true, y_pred2)

    # Perform paired t-test
    _, p_value = stats.ttest_rel(np.abs(y_true - y_pred1), np.abs(y_true - y_pred2))
    _, p_value2 = stats.ttest_rel(y_true - y_pred1, y_true - y_pred2)

    # Calculate variances
    var_a = variance1**2
    var_b = variance2**2

    # Calculate F statistic
    F = var_a / var_b
    df1 = len(y_true) - 1  # degrees of freedom for sample 1
    df2 = len(y_true) - 1  # degrees of freedom for sample 2

    # Calculate p-value
    p_value3 = 1 - stats.f.cdf(F, df1, df2) if var_a > var_b else stats.f.cdf(F, df1, df2)

    return mae1, bias1, mae2, bias2, variance1, variance2, p_value, p_value2, p_value3

# def get_scores(test, pia, hybrid, inx):    
#     
#     y1 = pia[:, inx].detach().cpu().numpy()
#     corr1 = round(pearsonr(y1, x1)[0], 2)
#     mae1 = np.mean(np.abs(y1 - x1))
#     bias1 = np.mean((y1 - x1))
#     std1 = np.std((y1 - x1))
#     x2 = test.detach().cpu().numpy()[:, inx] 
#     
#     corr2 = round(pearsonr(y2, x2)[0], 2)
#     mae2 = np.mean(np.abs(y2 - x2))
#     bias2 = np.mean((y2 - x2))
#     std2 = np.std((y2 - x2))
#     return  (corr1, corr2), (mae1,mae2), (bias1,bias2), (std1,std2)





    
PATH = '/home/gundogdu/Desktop/PIA_Histology/pia_model.pt'
model = torch.load(PATH)
N = 0.02
B = 2500
_, D_test2, T2_test2, v_test, clean = get_batch(B, noise_sdt=0)
clean = clean.detach().cpu().numpy()
noise_im = np.random.normal(0, N, (B, 16))
noise_re = np.random.normal(0, N, (B, 16))
noisy = np.sqrt((clean + noise_im)**2 + (clean + noise_re)**2)/np.sqrt(2)

test_tensor = 1000*torch.from_numpy(noisy).float()
test_tensor = test_tensor.cuda()
D_pia, T2_pia, v_pia = model.encode(test_tensor)
test = test_tensor.detach().cpu().numpy()/1000
D_hybrid, T2_hybrid, v_hybrid = hybrid_fit(test)




In [ ]:
for r in range(3):
    if r==0:
        x_image, y_image = v_test, v_hybrid
        title = ['V_ep', 'V st', 'V lu']
    elif r==1:
        x_image, y_image = D_test2, D_hybrid
        title = ['D ep', 'D st', 'D lu']
    elif r==2:
        x_image, y_image = T2_test2, T2_hybrid
        title = ['T2 ep', 'T2 st', 'T2 lu']
    for c in range(3):

        x = x_image.detach().cpu().numpy()[:,c] 
        y = y_image[:, c]

        corr = round(pearsonr(y, x)[0], 2)
        mae = np.mean(np.abs(y - x))
        bias = np.mean((y - x))
        std = np.std((y - x))
        print(f'{title[c]} {corr:.2f} {mae:.2f} {bias:.2f} {std:.2f}')
print('--------------------------')
for r in range(3):
    if r==0:
        x_image, y_image = v_test, v_pia
        title = ['V_ep', 'V st', 'V lu']
    elif r==1:
        x_image, y_image = D_test2, D_pia
        title = ['D ep', 'D st', 'D lu']
    elif r==2:
        x_image, y_image = T2_test2, T2_pia
        title = ['T2 ep', 'T2 st', 'T2 lu']
    for c in range(3):

        x = x_image.detach().cpu().numpy()[:,c] 
        y = y_image[:, c].detach().cpu().numpy()

        corr = round(pearsonr(y, x)[0], 2)
        mae = np.mean(np.abs(y - x))
        bias = np.mean((y - x))
        std = np.std((y - x))
        print(f'{title[c]} {corr:.2f} {mae:.2f} {bias:.2f} {std:.2f}')

In [ ]:
def get_scores(test, pia, hybrid, inx, B):
    x1 = test[:,inx].detach().cpu().numpy()
    y1 = pia[:, inx].detach().cpu().numpy()
    y2 = hybrid[:, inx]
    r_pia = pearsonr(x1, y1)[0]
    r_hybrid = pearsonr(x1, y2)[0]
    mae_pia, bias_pia, mae_hybrid, bias_hybrid, var_pia, var_hybrid, p_value, p_value2, p_value3  = compare_mae(x1, y1, y2)
    print(f"Paired t-test on absolute error - p-value: {p_value:.4f}")
    print(f"Paired t-test on bias - p-value: {p_value2:.4f}")
    print(f"F-test on variances - p-value: {p_value3:.4f}")
    print(f'Steiger\'s z-test for correlation strenghths of PIA and Hybrid p = {steigers_z_test(r_hybrid, r_pia, B, B)[1]}')

get_scores(T2_test2, T2_pia, T2_hybrid, 2, B)


In [ ]:
D, T2, v = model.encode(test_tensor)

for r in range(3):
    if r==0:
        x_image, y_image = v_test, v
        title = ['V_ep', 'V st', 'V lu']
        ylims = [(0,1), (0,1), (0,1)]
    elif r==1:
        x_image, y_image = D_test2, D
        title = ['D ep', 'D st', 'D lu']
        ylims = [(0.3, 0.7), (0.7, 1.7), (2.7, 3)]
    elif r==2:
        x_image, y_image = T2_test2, T2
        title = ['T2 ep', 'T2 st', 'T2 lu']
        ylims = [(20, 70), (40, 100), (500, 1000)]
    for c in range(1):
        fig, ax = plt.subplots(1,1, figsize=(10,10))
        x = x_image.detach().cpu().numpy()[:,c] 
        y = y_image[:, c].detach().cpu().numpy()
        nbins=300
        k = gaussian_kde([x,y])
        xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
        zi = k(np.vstack([xi.flatten(), yi.flatten()]))
        ax.pcolormesh(xi, yi, zi.reshape(xi.shape), cmap="hot", shading='auto')
        ax.scatter(x,y, color='white', s=4, alpha=0.5)

        #err = np.mean(np.abs(x_image.detach().cpu().numpy()[:,c]-y_image[:, c]))
        #corr = np.corrcoef(x_image.detach().cpu().numpy()[:,c],y_image[:, c])[0,1]
        ax.xaxis.set_tick_params(labelsize=20)
        ax.yaxis.set_tick_params(labelsize=20)
        #ax[r,c].set_title(fr'{title[c]}, MAE = {err:.3f}, $\rho$ = {corr:.3f}')
        ax.set_xlabel('true', fontsize=24)
        ax.set_ylabel('predicted', fontsize=24)


In [ ]:
# Speed Analysis
test_tensor, D_test2, T2_test2, v_test, clean = get_batch(10000, noise_sdt=0.01)
test = test_tensor.detach().cpu().numpy()


In [ ]:
start = time.time()
D, T2, v = hybrid_fit(test)
end = time.time()
print(f'Hybrid 1.0 takes {end - start} seconds')

In [ ]:
model2 = PIA(predictor_depth=2).float()#.to('cpu')
test_tensor = test_tensor.to('cuda')




In [ ]:
start = time.time()
D, T2, v = model2.encode(test_tensor) 
end = time.time()
print(f'Hybrid 2.0 takes {end - start} seconds')

In [ ]:
502/0.08

In [ ]:
test_tensor